# Webscraping for the Cincinnati Children's Hospital Medical Center LOVD 
The CCHMC LOVD page is still on the LOVD version 2 format, so it is a little bit more difficult to scrape than the LOVD3 sites. It is possible to see some information about the individual variants, but the clinical information is all obtained on a separate page for each individual variant. That said, I have to scrape one web page to find the subsequent pages, and then I have to request those pages and scrape the information from each of those. Additionally, the number of fields on those subsequent pages can vary, so I ended up putting the data into a dictionary and then putting a dash for the fields that were missing. Once the data from webscraping is obtained, I have to read in a file that gives the versions of transcripts that Biocommons has (because it will throw errors with the ones the CCHMC site gives) and then converts the cDNA information to genomic information. After that, the genomic information is used to populate fields that can be obtained in ClinVar and the output is put in a format that mirrors the ClinVar output as close as I was able to make it.   

The base page is [this](https://research.cchmc.org/LOVD2/variants.php?select_db=ACADM&action=search_all&order=Variant%2FDNA%2CASC&hide_col=&show_col=&limit=1000&search_pathogenic_=&search_Variant%2FDNA=&search_Variant%2FProtein=&search_Variant%2FRemarks=&search_Variant%2FdbSNP=&search_Patient%2FPatient%2FRace=&search_Patient%2FReference=&search_Patient%2FOrigin%2FEthnic=). 

You can find the transcript, the last updated date and the chromosome on this [page](https://research.cchmc.org/LOVD2/home.php?select_db=ACADM). 
 

For now, the first cell lists the genes of interest. This will probably be converted to a .py file and we will move the list of genes out to the command line so that you can include any genes of interest, but for now, it is the first cell. 

In [1]:
genes_for_metabolic_diseases = ["ACADM", "ACADS", "ACADVL", "CPT1A", "CPT2", "ETFA", "ETFB", "ETFDH", "HADH",
                               "HADHA", "HADHB", "SLC22A5", "SLC25A20"]
SCID_genes = ["CORO1A", "CD3Z", "ATM", "LIG4", "DOCK2", "PRKDC", "DCLRE1C", "CHD7", "NHEJ1", "GATA2",
            "TBX1", "IL2RG", "MTHFD1", "RAC2", "IGHM", "ADA", "IL7RA", "MTR", "CD3G", "BTK",
            "AK2", "JAK3", "RMRP", "STAT5B", "CD40LG", "CD3D", "RAG1", "SLC46A1", "ZAP70", "WAS",
            "CD3E", "RAG2", "DOCK8", "PNP", "DKC1", "PTPRC", "FOXN1", "NBN", "BLNK"]

In [2]:
import os
ncbi_api_key = os.environ.get('ncbi_api_key') # I don't think this is doing anything
# this should theoretically speed it up to 10 per second instead of 3 per second
# https://github.com/biocommons/bioutils

import pandas as pd

import re
from bs4 import BeautifulSoup
import warnings 
import requests 
import json 
import hgvs.normalizer
import hgvs.variantmapper
import hgvs.dataproviders.uta
import hgvs.assemblymapper
import hgvs.parser
import hgvs.validator
import hgvs.exceptions
import pandas as pd
import numpy as np
hdp = hgvs.dataproviders.uta.connect()
hp = hgvs.parser.Parser()
hn = hgvs.normalizer.Normalizer(hdp)
am = hgvs.assemblymapper.AssemblyMapper(hdp, assembly_name='GRCh37', alt_aln_method='splign', replace_reference=True)
vr = hgvs.validator.Validator(hdp=hdp)

from datetime import datetime

output_directory = "All_ClinVar_output_files/"
os.makedirs(output_directory+"ClinVar/Not_Valid_Annotations/", exist_ok=True)

from datetime import datetime

ncbi_api_key = os.environ.get('ncbi_api_key')
output_directory = "output_files/"
os.makedirs(output_directory+"CCHMC_LOVD/", exist_ok=True)

In [9]:
transcripts_df = pd.read_csv("Transcript_Info_For_Dictionaries.csv", index_col = 0) #, header = None, skiprows=1, index_col = 0
transcript_dictionary = dict(zip(list(transcripts_df["Transcript"].values), list(transcripts_df["Transcript_ID_Full"].values)))
gene_to_transcript_dict = dict(zip(list(transcripts_df["Gene"].values), list(transcripts_df["Transcript_ID_Full"].values)))

def web_scrape_cchmc(disease, list_of_genes):
    for gene in list_of_genes:
        result_list = []
        home_link = "https://research.cchmc.org/LOVD2/home.php?select_db="+gene
        response = requests.get(home_link)
        response_html = response.content.decode("utf-8")
        bs_format = BeautifulSoup(response_html, "html.parser")
        potential_error_message = bs_format.select('th')[0].get_text()
        if potential_error_message == 'Error: Init':
            print("The CCHMC LOVD2 database does not contain any variants for the gene "+gene+".", sep = "")
            # I could change this out for a warning, but the CCHMC database doesn't contain any info for the majority of the genes
            # so I thought this would be more appropriate to standard output instead of a warning to standard error
            continue

        updated_date = "-"
        chromosome = "-"
        transcript = "-"
        transcript_new = "-"
        genomic_ref = "-" # This is still referring to the gene so I don't think we will use it, but I am grabbing it for just in case
        for term in bs_format.select('th'):
            term_text = term.get_text()

            if term_text == "Last update" :
                updated_date = term.parent.select('td')[0].get_text()
            elif term_text == "Chromosome Location":
                chromosome_search_result = re.search(r"([0-9,X,Y,M,T]+)[p|q]\d+", term.parent.select('td')[0].get_text())
                if chromosome_search_result != None:
                    chromosome = chromosome_search_result.group(1)
            elif "Transcript" in term_text:
                transcript = term.parent.select('td')[0].get_text()
            elif "Genomic" in term_text:
                genomic_ref = term.parent.select('td')[0].get_text()
        transcript_seach = re.search(r"NM_\d+", transcript)
        if transcript_seach != None:
            transcript_parent = transcript_seach.group(0)
            transcript_new = transcript_dictionary[transcript_parent]
        ##
        ## The ones that had no transcript listed on the home page only have one transcript
        ## so I am going to use the dictionary to look up the transcript for those ones
        ## 
        if transcript_new == "-":
            transcript_new = gene_to_transcript_dict[gene]

        link = "https://research.cchmc.org/LOVD2/variants.php?select_db="+gene+"&action=search_all&order=Variant%2FDNA%2CASC&hide_col=&show_col=&limit=1000&search_pathogenic_=&search_Variant%2FDNA=&search_Variant%2FProtein=&search_Variant%2FRemarks=&search_Variant%2FdbSNP=&search_Patient%2FPatient%2FRace=&search_Patient%2FReference=&search_Patient%2FOrigin%2FEthnic="
        response = requests.get(link)
        if response.status_code == 200:
            response_html = response.content.decode("utf-8")
            bs_format = BeautifulSoup(response_html, "html.parser")
            potential_error_message = bs_format.select('img[src="./gfx/header_variant_listings.png"]')[0].parent.parent.select('td')[0].get_text().strip()
            if "currently no public variants" in potential_error_message:
                print("The CCHMC LOVD2 database does not contain any variants for the gene "+gene+".", sep = "")
                continue
                # This should not happen because it should have happened with the first link if this were the case. 

            links = []
            anchor_data = bs_format.select('a[class="data"]')
            for anchor in anchor_data:
                links.append("https://research.cchmc.org"+anchor.attrs['href'])
            result_list = []
            
            for link in links:
                response = requests.get(link)
                if response.status_code != 200:
                    continue
                    # I am not going to write a warning statement for any of these 
                    # because if they fail we only miss out on a single variant
                response_html = response.content.decode("utf-8")
                bs_format = BeautifulSoup(response_html, "html.parser")
                ths = bs_format.select('th[valign="top"]')
                individual_results = []
                individual_labels = []
                for th in ths:
                    # This can get all of the column values for the things we care about in a list format
                    # Unfortunately, some variables are missing for some variants
                    individual_results.append(th.parent.td.get_text().replace(u'\xa0', u' ').replace('(View in UCSC Genome Browser, Ensembl)', ''))
                for label in bs_format.select('th[valign="top"]'):
                    # This will be the labels that correspond with the values obtained above. 
                    individual_labels.append(label.get_text().replace(u'\xa0', u' '))
                # I have the values and their labels in separate lists at this point. 
                # Now I am going to put them into a dictionary so that I can have key, value pairs
                # and I can put in a dash for anything that isn't present
                individual_dictionary = dict(zip(individual_labels, individual_results))
                if "Race" not in individual_dictionary:
                    individual_dictionary["Race"] = "-"
                if "Reference" not in individual_dictionary:
                    individual_dictionary["Reference"] = "-"
                if "Ethnic origin" not in individual_dictionary:
                    individual_dictionary["Ethnic origin"] = "-"
                if "Submitter" not in individual_dictionary:
                    individual_dictionary["Submitter"] = "-"
                if "Allele" not in individual_dictionary:
                    individual_dictionary["Allele"] = "-"
                if "Reported pathogenicity" not in individual_dictionary:
                    individual_dictionary["Reported pathogenicity"] = "-"
                if "Concluded pathogenicity" not in individual_dictionary:
                    individual_dictionary["Concluded pathogenicity"] = "-"
                if "DNA change" not in individual_dictionary:
                    individual_dictionary["DNA change"] = "-"
                if "Protein" not in individual_dictionary:
                    individual_dictionary["Protein"] = "-"
                if "Variant remarks" not in individual_dictionary:
                    individual_dictionary["Variant remarks"] = "-"
                if "dbSNP ID" not in individual_dictionary:
                    individual_dictionary["dbSNP ID"] = "-"

                # Now I am going to convert the pathogenicity to the ClinVar terms
                pathogenicity_clinvar = "-"
                if individual_dictionary["Reported pathogenicity"] == "No known pathogenicity":
                    pathogenicity_clinvar = "Benign"
                elif individual_dictionary["Reported pathogenicity"] == "Probably no pathogenicity":
                    pathogenicity_clinvar = "Likely benign"
                elif individual_dictionary["Reported pathogenicity"] == "Unknown":
                    pathogenicity_clinvar = "Uncertain significance"
                elif individual_dictionary["Reported pathogenicity"] == "Probably pathogenic":
                    pathogenicity_clinvar = "Likely pathogenic"
                elif individual_dictionary["Reported pathogenicity"] == "Pathogenic":
                    pathogenicity_clinvar = "Pathogenic"


                ###
                ###  Now I need to see if I can use biocommons to get the genomic annotation
                ###  I don't need to keep the rest of these in a dictionary because I can save them as variables
                ###          
                hgvs_transcript = "-"
                Genomic_annotation = "-"
                variant = individual_dictionary["DNA change"].strip() # This should always be present, even if it is a dash
                if transcript_new != "-" and variant != "-":
                    hgvs_transcript = transcript_new+":"+variant
                elif transcript != "-" and variant != "-":
                    # if there is no new transcript, give it a test with the old one
                    hgvs_transcript = transcript+":"+variant
                    # Leave it as a dash if the DNA change is a dash, which is common
                if hgvs_transcript != "-":
                    try:
                        Genomic_annotation = str(am.c_to_g(hp.parse_hgvs_variant(hgvs_transcript)))
                    except hgvs.exceptions.HGVSError:
                        Genomic_annotation = "-"
                Position_g_start = "-"
                Position_g_stop = "-"
                Ref_allele = "-"
                Alt_allele = "-"
                Var_Type = "-"
                Chr_Accession = "-"
                Var_Length = "-"


                if Genomic_annotation != "-":
                    # If we have the genomic annotation, we can use this to populate the fileds that are empty
                     # If we can't get a genomic annotation, then all the other variables will be left as a dash
                    chrom_accession_search = re.search(r"NC_\d+\.\d+", Genomic_annotation)
                    if chrom_accession_search != None:
                        Chr_Accession =chrom_accession_search.group(0)
                    if ">" in Genomic_annotation:
                        Var_Type = "single nucleotide variant"
                    elif "delins" in Genomic_annotation:
                        Var_Type = "Indel"
                    elif "del" in Genomic_annotation:
                        Var_Type = "Deletion"
                    elif "ins" in Genomic_annotation:
                        Var_Type = "Insertion"
                    elif "dup" in Genomic_annotation:
                        Var_Type = "Duplication"
                    elif "inv" in Genomic_annotation:
                        Var_Type = "Inversion"
                    # These have all been normalized by biocommons, so this should work. 

                    pos_start_search = re.search(r"g.(\d+)", Genomic_annotation)
                    if pos_start_search != None:
                        Position_g_start = pos_start_search.group(1)
                    if Var_Type == "single nucleotide variant":
                        Position_g_stop = Position_g_start
                        Var_Length = 1
                        ref_allele_search = re.search(r"g.\d+([A,C,T,G])>([A,C,T,G])", Genomic_annotation)
                        if ref_allele_search != None:
                            Ref_allele = ref_allele_search.group(1)
                            Alt_allele = ref_allele_search.group(2)
                    elif Var_Type == "Indel":
                        # Var_Length for indels is inconsistent in ClinVar, so I will leave these as a dash
                        alt_allele_search = re.search(r"ins([A,C,T,G]+)", Genomic_annotation)
                        if alt_allele_search != None:
                            Alt_allele = alt_allele_search.group(1)
                        multiple_location_search = re.search(r"\d+_\d+", Genomic_annotation)
                        if multiple_location_search == None:
                            Position_g_stop = Position_g_start      
                        else:
                            stop_search = re.search(r"_(\d+)del", Genomic_annotation)
                            if stop_search != None:
                                Position_g_stop = stop_search.group(1)
                    elif Var_Type == "Insertion":
                        # Reference will always be a dash
                        # I think the stop should always be start plus one, but in case it is funky I am gong to use the regex anyway
                        stop_search = re.search(r"_(\d+)[a-z]+", Genomic_annotation)
                        if stop_search != None:
                            Position_g_stop = stop_search.group(1)
                        # Ref allele is always a dash, and we can get the Alt allele from the end of this 
                        alt_allele_search = re.search(r"ins([A,C,T,G]+)", Genomic_annotation)
                        if alt_allele_search != None:
                            Alt_allele = alt_allele_search.group(1)
                        if Alt_allele != "-":
                            Var_Length = len(Alt_allele)
                    elif Var_Type == "Duplication" or Var_Type == "Deletion" or Var_Type == "Inversion":
                        # Can't get reference or alternate for either of these
                        multiple_location_search = re.search(r"\d+_\d+", Genomic_annotation)
                        if multiple_location_search == None:
                            Position_g_stop = Position_g_start 
                            Var_Length = 1
                        else:
                            stop_search = re.search(r"_(\d+)[a-z]+", Genomic_annotation)
                            if stop_search != None:
                                Position_g_stop = stop_search.group(1)
                                Var_Length = int(Position_g_stop) - int(Position_g_start) + 1

                # Genomic annotation goes in twice for the genomic annotation and HGVS normalized genomic annotation (to match ClinVar output format)
                # gene will also be in here twice for Affected Genes and Gene Symbol
                individual_variant_info = ["GRCh37", chromosome, Position_g_start, Position_g_stop, Ref_allele,
                                          Alt_allele, Genomic_annotation, Genomic_annotation, Var_Type, Var_Length,
                                          pathogenicity_clinvar, "-", individual_dictionary["Allele"],
                                          "-", gene, gene, "-", transcript_new, variant, 
                                          hgvs_transcript, "-", individual_dictionary["Protein"], "-", Chr_Accession,
                                          "-", "-", "-", "CCHMC_LOVD2", "-", "-", "-", individual_dictionary["Submitter"], 
                                          updated_date, individual_dictionary["Concluded pathogenicity"],
                                           individual_dictionary["Reported pathogenicity"], individual_dictionary["Race"],
                                          individual_dictionary["Ethnic origin"], individual_dictionary["Reference"], 
                                          individual_dictionary["Variant remarks"], transcript, individual_dictionary["dbSNP ID"],
                                          genomic_ref]
                result_list.append(individual_variant_info)
        gene_df = pd.DataFrame(result_list)
        # The added columns starting with "CCHMC Reported Pathogenicity" are all new to this one and not in ClinVar
        gene_df.columns = ["Genome Assembly", "Chr", "Position Start", "Position Stop", "Ref", "Alt",
                           "Genomic Annotation", "HGVS Normalized Genomic Annotation", "Variant Type",
                            "Variant Length", "Pathogenicity", "Disease", "Genetic Origin", "Inheritance Pattern",
                            "Affected Genes" , "Gene Symbol", "Compound Het Status", "Transcript",
                            "Transcript Notation", "HGVS Transcript Notation","Protein Accession",
                            "Protein Notation", "HGVS Protein Annotation", "Chr Accession", "VCF Pos",
                            "VCF Ref", "VCF Alt", "Database", "ClinVar Accession", "Review Status",
                            "Star Level", "Submitter", "Edited Date", "CCHMC Reported Pathogenicity", 
                           "CCHMC Concluded Pathogenicity", "Race", "Ethnic Origin", "Reference Paper", 
                          "Variant Remarks", "CCHMC Reported Transcript", "dbSNP ID", "Genomic Accession"]
        gene_df.to_csv(output_directory+"CCHMC_LOVD/"+disease+"_"+gene+"_CCHMC_results.csv")
        print("Finished obtaining variant information for gene "+gene+" in the CCHMC database.", sep = "")

In [4]:
web_scrape_cchmc("Metabolic_Diseases", genes_for_metabolic_diseases)

Finished obtaining variant information for gene ACADM in the CCHMC database.
The CCHMC LOVD2 database does not contain any variants for the gene ACADS.
Finished obtaining variant information for gene ACADVL in the CCHMC database.
The CCHMC LOVD2 database does not contain any variants for the gene CPT1A.
The CCHMC LOVD2 database does not contain any variants for the gene CPT2.
The CCHMC LOVD2 database does not contain any variants for the gene ETFA.
The CCHMC LOVD2 database does not contain any variants for the gene ETFB.
The CCHMC LOVD2 database does not contain any variants for the gene ETFDH.
The CCHMC LOVD2 database does not contain any variants for the gene HADH.
The CCHMC LOVD2 database does not contain any variants for the gene HADHA.
The CCHMC LOVD2 database does not contain any variants for the gene HADHB.
The CCHMC LOVD2 database does not contain any variants for the gene SLC22A5.
The CCHMC LOVD2 database does not contain any variants for the gene SLC25A20.


In [7]:
web_scrape_cchmc("SCID", SCID_genes)

The CCHMC LOVD2 database does not contain any variants for the gene CORO1A.
The CCHMC LOVD2 database does not contain any variants for the gene CD3Z.
The CCHMC LOVD2 database does not contain any variants for the gene ATM.
The CCHMC LOVD2 database does not contain any variants for the gene LIG4.
The CCHMC LOVD2 database does not contain any variants for the gene DOCK2.
The CCHMC LOVD2 database does not contain any variants for the gene PRKDC.
The CCHMC LOVD2 database does not contain any variants for the gene DCLRE1C.
The CCHMC LOVD2 database does not contain any variants for the gene CHD7.
The CCHMC LOVD2 database does not contain any variants for the gene NHEJ1.
The CCHMC LOVD2 database does not contain any variants for the gene GATA2.
The CCHMC LOVD2 database does not contain any variants for the gene TBX1.
Finished obtaining variant information for gene IL2RG in the CCHMC database.
The CCHMC LOVD2 database does not contain any variants for the gene MTHFD1.
The CCHMC LOVD2 database 

In [10]:
failed_genes_list = ["IL2RG", "WAS"]
web_scrape_cchmc("SCID", failed_genes_list)

Finished obtaining variant information for gene IL2RG in the CCHMC database.
Finished obtaining variant information for gene WAS in the CCHMC database.
